In [9]:
from arlyn import Scale
import re

arlyn_address = 'ip-address:port'

async def get_weight_string(scale_address):
    '''
    Retrieve the weight string from the scale 

    Args:
        scale_address: The serial or TCP address of the scale (string)

    Returns:
        The weight string from the scale (string)
        ex: '03/31/2024 17:28:08, 0.00 kg'
    '''
    async with Scale(scale_address) as scale:
        return await scale.print_weight()
    
async def toggle_unit(scale_address):
    '''
    Toggles the unit change command
    kg -> g -> oz -> lb ->

    Args:
        scale_address: The serial or TCP address of the scale (string)

    Returns:
        nothing
    '''
    async with Scale(scale_address) as scale:
        await scale.toggle_unit()

async def zero_scale(scale_address):
    '''
    Sets the current weight on the scale to zero

    Args:
        scale_address: The serial or TCP address of the scale (string)

    Returns:
        nothing
    '''
    async with Scale(scale_address) as scale:
        await scale.zero_scale()

async def get_json(scale_address):
    '''
    Retrieve a JSON string of the scales current status

    Args:
        scale_address: The serial or TCP address of the scale (string)

    Returns:
        JSON string of current scale status
        ex: '{"bSIndi":true,"sNet":"","sUnit":"kg","sWeight":"0.00","_CMD_ACK":"ACK","_id":1,"_name":"*W","_rSNo":42,"_sSNo":18}'
    '''
    async with Scale(scale_address) as scale:
        return await scale.get_json()
    
async def get_weight_dict(scale_address):
    '''
    Retrieve the weight string from the scale, then extracts the information into a dictionary
    Converts weight value into a float

    Args:
        scale_address: The serial or TCP address of the scale (string)

    Returns:
        The weight string from the scale (dict)
        ex: {'date': '03/31/2024', 'time': '17:35:55', 'weight': 0.0, 'unit': 'kg'}
    '''
    weight_string = await get_weight_string(scale_address)
    date = re.search(r"\d{2}/\d{2}/\d{4}",weight_string).group()
    time = re.search(r"\d{2}:\d{2}:\d{2}",weight_string).group()
    weight = re.search(r", .* ",weight_string).group()[2:-1]
    unit = re.search(r"[a-zA-Z]{1,2}",weight_string).group()
    return {"date":date,"time":time,"weight":float(weight),"unit":unit}

async def change_unit(scale_address, desired_unit):
    '''
    Toggles through units until scale is set to desired unit
    kg -> g -> oz -> lb ->

    Args:
        scale_address: The serial or TCP address of the scale (string)
        desired_unit: The unit to change the scale to (string)

    Returns:
        nothing
    '''
    units = ('kg','g','oz','lb')
    assert desired_unit in units, (
    "desired unit must be one of the following: kg, g, oz, lb")

    unit = await get_weight_dict(scale_address)
    unit = unit['unit']

    if desired_unit != unit:
        toggle_count = units.index(desired_unit) - units.index(unit) + len(units)
        for i in range(toggle_count):
            await toggle_unit(scale_address)
        unit = await get_weight_dict(scale_address)
        unit = unit['unit']
        print(f"unit set to: {unit}")
    else:
        print(f"unit was already set to: {unit}")

await change_unit(arlyn_address,'lb')
print(await get_weight_dict(arlyn_address))

unit was already set to: lb
{'date': '04/01/2024', 'time': '17:18:31', 'weight': 0.0, 'unit': 'lb'}
